### Imports

In [18]:
### Standard imports
import os
import pickle
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

from glob import glob
from os.path import join, isfile
from random import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU, ReLU
from tqdm import tqdm_notebook as tqdm

tf.random.set_seed(5)
random.seed(5)
np.random.seed(5)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

root = "/Users/pstetz/Desktop/confidential/.project"

### Helpers

In [19]:
def save_history(history_path, data):
    if isfile(history_path):
        pd.concat([pd.read_csv(history_path), pd.DataFrame(data)]).to_csv(history_path, index=False)
    else:
        pd.DataFrame(data).to_csv(history_path, index=False)

### Model

<div hidden>
def _load_model():
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
    activation = relu
    use_bias = False
    def layer_a(dim):
        return layers.Dense(dim, activation=activation, use_bias=use_bias)
    
    info_input = keras.Input(shape=(130,), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.MaxPool3D(),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Flatten()
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
#     for dim in (130, 130, 130):
#         info_s.append(layer_a(dim)(info_s[-1]))

    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    x_s = [x_0]
#     for dim in (146, 146, 146, 146, 128, 64, 32):
    for dim in (258, 258, 128, 128, 64, 32):
        x_s.append(layer_a(dim)(x_s[-1]))
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 1e-4
    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=8e-2, decay=learning_rate/30),
      loss={"bold_signal": "mse"},
      loss_weights=[1.])
    return model
</div>

In [15]:
def _load_model():
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
    activation = relu
    use_bias = False
    def layer_a(dim):
        return layers.Dense(dim, activation=activation, use_bias=use_bias)
    
    info_input = keras.Input(shape=(130,), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Flatten()
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    x_s = [x_0]
    for dim in (258, 258, 128, 128, 64, 32):
        x_s.append(layer_a(dim)(x_s[-1]))
        
    x = keras.layers.Reshape((num_features*num_preds,1))(inp)
    x = keras.layers.Conv1D(32,num_preds,strides=num_preds, activation='elu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(24,1, activation='elu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(16,1, activation='elu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(4,1, activation='elu')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Reshape((num_features*4,1))(x)
    x = keras.layers.AveragePooling1D(2)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.BatchNormalization()(x)
    out = keras.layers.Dense(1, activation='sigmoid')(x)
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 1e-4
    model.compile(
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=8e-2, decay=learning_rate/30),
        loss={"bold_signal": "mse"}, loss_weights=[1.]
    )
    return model

if "model" in locals(): del model
model = _load_model()

### Setup

In [16]:
history_path = join(root, "history.csv")
training_path = join(root, "training")
log_dir = join(root, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

train_batch  = 64
input_batch  = train_batch * 2
num_epoches = 2

if isfile(history_path):
    os.remove(history_path)

### Training

In [17]:
bold_signal, prev_volume, next_volume, info = [], [], [], []

for _ in range(4):
    training = glob(join(training_path, "*", "*"))
#     shuffle(training)
    for i, batch_path in enumerate(tqdm(training)):
        if not isfile(join(batch_path, "norm_info.npy")):
            print("%s not found" % batch_path)
            continue
        bold_signal.extend(np.load(join(batch_path, "pred.npy")))
        prev_volume.extend(np.load(join(batch_path, "norm_prev.npy")))
        next_volume.extend(np.load(join(batch_path, "norm_next.npy")))
        info.extend(np.load(join(batch_path, "norm_info.npy"), allow_pickle=True))
        if (i + 1) % input_batch == 0:
            batch = {
                "prev": np.array(prev_volume),
                "next": np.array(next_volume),
                "info": np.array(info, dtype=np.float32)
            }
            bold_signal = np.array(bold_signal)
            history = model.fit(
                batch, bold_signal,
                epochs=num_epoches, batch_size=train_batch, verbose=True,
                callbacks=[tensorboard_callback],
                shuffle=True # use when randomly selecting batches
            )
            _std = np.std(bold_signal).round(3)
            _loss = history.history["loss"][-1].round(3)
            assert not np.isnan(_loss), batch_path
            _mean = np.add(batch["prev"][:, 4, 4, 4, 0], batch["next"][:, 4, 4, 4, 0]) / 2
            mean_loss = np.sum(np.square(np.subtract(_mean, bold_signal))) / len(bold_signal)
            print(_loss, mean_loss)
            data = {
                "std": [_std] * num_epoches,
                "loss": history.history["loss"],
                "mean_loss": mean_loss
            }
            save_history(history_path, data)
            bold_signal, prev_volume, next_volume, info = [], [], [], []

/Users/pstetz/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Train on 16384 samples
Epoch 1/2
16384/16384 [==============================] - 8s 482us/sample - loss: 1.5247
Epoch 2/2
16384/16384 [==============================] - 9s 519us/sample - loss: 1.0008
1.001 0.8104316592216492
Train on 16384 samples
Epoch 1/2
16384/16384 [==============================] - 8s 517us/sample - loss: 0.9785
Epoch 2/2
16384/16384 [==============================] - 7s 437us/sample - loss: 0.9693
0.969 0.7196229696273804
Train on 16384 samples
Epoch 1/2
16384/16384 [==============================] - 7s 436us/sample - loss: 0.9663
Epoch 2/2
16384/16384 [==============================] - 7s 437us/sample - loss: 0.9587
0.959 0.8649934530258179
Train on 16384 samples
Epoch 1/2
16384/16384 [==============================] - 7s 404us/sample - loss: 0.9714
Epoch 2/2
16384/16384 [==============================] - 9s 536us/sample - loss: 0.9716
0.972 0.8284645080566406
Train on 16384 samples
Epoch 1/2
16384/16384 [==============================] - 7s 424us/sample - loss: 

16384/16384 [==============================] - 5s 329us/sample - loss: 0.8443
0.844 0.7333552837371826


KeyboardInterrupt: 

### Try double model (lgbm + CNN)

In [ ]:
lgbm_model = 

In [5]:
model.save("/Volumes/hd_4tb/results/3_more_filters.h5")